In [2]:
import polars as pl
import numpy as np

In [3]:
df = pl.scan_parquet('./Datasets/crsp')
df.head().collect_schema()

Schema([('PERMNO', Int64),
        ('date', String),
        ('NAMEENDT', String),
        ('SHRCD', Int64),
        ('EXCHCD', Int64),
        ('NCUSIP', String),
        ('TICKER', String),
        ('COMNAM', String),
        ('SHRCLS', String),
        ('PRIMEXCH', String),
        ('TRDSTAT', String),
        ('SECSTAT', String),
        ('PERMCO', Int64),
        ('CUSIP', String),
        ('DCLRDT', String),
        ('DLAMT', Float64),
        ('DLPDT', String),
        ('DLSTCD', Float64),
        ('PAYDT', String),
        ('RCRDDT', String),
        ('SHRFLG', Float64),
        ('DISTCD', Float64),
        ('DIVAMT', Float64),
        ('FACPR', Float64),
        ('FACSHR', Float64),
        ('ACPERM', Float64),
        ('ACCOMP', Float64),
        ('SHRENDDT', String),
        ('NWPERM', Float64),
        ('DLRETX', String),
        ('DLPRC', Float64),
        ('DLRET', String),
        ('BIDLO', Float64),
        ('ASKHI', Float64),
        ('PRC', Float64),
        ('VOL', F

In [4]:
# efficient daily log returns for Apple
aapl_permno = df.filter(pl.col('TICKER') == 'AAPL') \
    .select('PERMNO') \
    .unique() \
    .collect() \
    .item()

log_returns = df.filter(pl.col('PERMNO') == aapl_permno) \
    .select(['PERMNO', 'PRC']) \
    .with_columns(
        (pl.col('PRC')/pl.col('PRC').shift(1)) \
            .log()
            .alias('LOG_RET')
        ) \
    .collect()

df_crsp = df

In [42]:
# compustat rolling sum
df_cstat = pl.scan_parquet('./Datasets/compustat/fundamentals_quarterly_all')
cstat_clean = df_cstat.select(['gvkey', 'datadate', 'gsector', 'tic', 'cusip', 'conm', 'cik', 'fyearq', 'fqtr', 
                 'rdq', 'indfmt', 'datafmt', 'consol', 'curcdq', 'costat', 'prccq', 'cshoq',
                 'ajexq', 'mkvaltq', 'atq', 'actq', 'cheq', 'rectq', 'invtq', 'ppentq',
                 'ltq', 'lctq', 'dlcq', 'dlttq', 'apq', 'txdbq','seqq', 'ceqq', 'pstkq',
                 'saleq', 'cogsq', 'xsgaq', 'xrdq', 'dpq', 'xintq','oiadpq', 'piq', 'txtq', 
                 'niq', 'ibq','epsfxq', 'oancfy', 'capxy', 'dvy', 'aqcy', 'epspxq', 'opepsq'])
cstat_clean.filter(pl.col('gsector').is_not_null()).tail().collect()

gvkey,datadate,gsector,tic,cusip,conm,cik,fyearq,fqtr,rdq,indfmt,datafmt,consol,curcdq,costat,prccq,cshoq,ajexq,mkvaltq,atq,actq,cheq,rectq,invtq,ppentq,ltq,lctq,dlcq,dlttq,apq,txdbq,seqq,ceqq,pstkq,saleq,cogsq,xsgaq,xrdq,dpq,xintq,oiadpq,piq,txtq,niq,ibq,epsfxq,oancfy,capxy,dvy,aqcy,epspxq,opepsq
str,date,str,str,str,str,str,i32,i16,date,str,str,str,str,str,"decimal[24,12]","decimal[18,4]","decimal[24,12]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]","decimal[18,4]"
"""266160""",2025-12-31,"""40""","""CZWI""","""174903104""","""CITIZENS COMMUNITY BANCORP""","""0001367859""",2025,4,2026-01-26,"""INDL""","""STD""","""C""","""USD""","""A""",17.820000000000,9.6170,1.000000000000,171.3749,1781.7550,null,118.8530,1317.9240,5.8110,16.3570,1593.8160,null,null,null,1524.0990,0.0000,187.9390,187.9390,0.0000,24.4630,8.9060,7.1550,null,null,null,8.4020,4.8850,0.6140,4.2710,4.2710,0.4400,null,null,null,null,0.4400,0.4400
"""266214""",2025-12-31,"""40""","""CNS""","""19247A100""","""COHEN & STEERS INC""","""0001284812""",2025,4,2026-01-22,"""INDL""","""STD""","""C""","""USD""","""A""",62.780000000000,51.0040,1.000000000000,3202.0311,null,null,null,null,null,null,null,null,null,null,null,null,562.0000,562.0000,0.0000,143.8030,100.9580,null,null,2.5350,0.0000,40.3100,42.2880,11.5850,34.8790,34.8790,0.6800,null,null,null,null,0.6800,0.6800
"""294524""",2025-12-31,"""15""","""LYB""","""N53745100""","""LYONDELLBASELL INDUSTRIES NV""","""0001489393""",2025,4,2026-01-30,"""INDL""","""STD""","""C""","""USD""","""A""",43.300000000000,322.0000,1.000000000000,13942.6000,34003.0000,10868.0000,3449.0000,2517.0000,3533.0000,17347.0000,23796.0000,6129.0000,814.0000,13451.0000,2694.0000,2316.0000,10082.0000,10082.0000,0.0000,7091.0000,6335.0000,397.0000,33.0000,385.0000,132.0000,-26.0000,-142.0000,-8.0000,-142.0000,-136.0000,-0.4300,2262.0000,1878.0000,1764.0000,null,-0.4300,-0.2600
"""317264""",2025-12-31,"""10""","""LPG""","""Y2106R110""","""DORIAN LPG LTD""","""0001596993""",2025,3,2026-02-05,"""INDL""","""STD""","""C""","""USD""","""A""",24.340000000000,42.7440,1.000000000000,1040.3890,1777.7390,392.2860,294.4920,73.3500,2.2510,1328.0300,692.1100,162.4620,145.5590,528.0680,6.6910,0.0000,1085.6290,1085.6290,0.0000,120.6500,42.3170,10.7800,null,16.2420,6.8300,51.3110,47.1890,0.0000,47.1890,47.1890,1.1100,127.9960,29.6650,75.2490,0.0000,1.1100,1.1100
"""326688""",2025-12-31,"""20""","""NVT""","""G6700G107""","""NVENT ELECTRIC PLC""","""0001720635""",2025,4,2026-02-06,"""INDL""","""STD""","""C""","""USD""","""A""",101.970000000000,161.3600,1.000000000000,16453.8792,6851.9000,1639.6000,237.5000,693.0000,null,434.5000,3121.7000,1003.5000,13.8000,1546.0000,358.9000,232.0000,3730.2000,3730.2000,0.0000,1066.7000,677.8000,219.7000,21.3000,null,19.1000,169.2000,156.3000,40.5000,118.8000,115.8000,0.7100,465.2000,93.3000,130.4000,975.7000,0.7200,0.7400


In [ ]:
# earnings TTM (trailing twelve month) rolling sum: cont.
result = cstat_clean.select(['tic', 'gvkey', 'epspxq']) \
    .with_columns(
        pl.col("epspxq")
        .cast(pl.Float64)
        .rolling_sum(window_size=4)
        .over('gvkey')
        .alias('TTM')
    )

result.collect()

tic,gvkey,epspxq,TTM
str,str,"decimal[18,4]",f64
"""ADCT.1""","""001013""",0.0200,null
"""SERV.1""","""001082""",-0.2000,null
"""AIM.1""","""001173""",0.1700,null
"""IDAI.""","""001183""",-0.1900,null
"""ABS.1""","""001240""",0.3500,null
…,…,…,…
"""FEX""","""177422""",null,null
"""PIN""","""179400""",null,null
"""UTG""","""260849""",null,null
